In [3]:
from PIL import Image
import numpy as np
import random
from qiskit import *

Open whatever image you have handy

In [4]:
img = Image.open('pic.png')

We are going to pull out all the RGB values of all the pixels in this image and mess around with them. So here's a quick function that will put them back together into an image.

In [5]:
def show_image(pixels):
    # creates and shows an image with the given pixel values
    new_img = Image.new('RGB',(width,height))  
    for x in range(width):
        for y in range(height):
            new_img.load()[x,y] = pixels[x,y]
    new_img.show()

First off, we pull out the RGB (and alpha) values of the pixels and put them in a dictionary called `pixels`. Then when we call `pixels[x,y]`, this will give use the RGB values for the coordinate at position $(x,y)$.

In [6]:
# load image
height = img.height
width = img.width

# get all pixel data out
pixels = {}
for x in range(width):
    for y in range(height):
        pixels[x,y] =  img.getpixel((x,y)) 

We'll remove the transparency in future steps. In preparation for that, we should make completely transparent pixels into white pixels.

In [7]:
for pos in pixels:
    if pixels[pos][3]==0:
        pixels[pos] = (255,255,255,255)

Pixels are described by three values: the RGB values. Qubits are also described by three values: the X, Y and Z expectation values (which are also the cartesian coordinates of the Bloch sphere). So this gives a natural way to encode pixel data into qubits.

One complication is that the all the channels for RGB can take extreme values at once. For example, you can have blue fully on, and red and green fully off. For qubits this would be mapped to X, Y and Z values of $(1,-1,-1)$, which lies outside of the Bloch sphere. It would violate the uncertainty principle, since it would mean that the outcome would be certain for measurements in all Pauli bases.

For this reason, we must normalize rgb values to put them on the surface of the Bloch sphere (we could also let them live inside the Bloch sphere, but I can't be bothered with mixed states).

In [8]:
# we associate r, g and b values with X, Y and Z expectation values
# this places a constraint on allowed rgb values, since the X, Y and Z must correspond to a point on the Bloch sphere
for pos in pixels:
    
    # convert rgb to pauli expectation values
    X = pixels[pos][0]/255-0.5
    Y = pixels[pos][1]/255-0.5
    Z = pixels[pos][2]/255-0.5
    
    # renormalize to surface of Bloch sphere
    N = np.sqrt( X**2 + Y**2 + Z**2 )
    X = X/N
    Y = Y/N
    Z = Z/N
    
    # set pixels to new rgb values (also remove all transparency)
    pixels[pos] = ( int( 255*(X+0.5)), int( 255*(Y+0.5)), int( 255*(Z+0.5)), 255 )
    
show_image(pixels)

For this encoding, the $\left|0\right\rangle$ state corresponds to blue. A fully blue image is therefore our blank slate.

Taking such a blank image, we could try to copy our original image with CNOTs. This would copy the Z coordinate, but result in the X and Y coordinates being set to zero for both images.

For such a simple effect, there's no need to use Qiskit to see what happens. Here's what both copies would look like.

In [9]:
# let's see what happen if a process causes the X and Y expectation values to go to zero
# this could be because the pixels are entangled with something else, or because of decoherence
decohered_pixels = {}
for pos in pixels:
    decohered_pixels[pos] = ( pixels[pos][0], 128, 128, 255 ) # 128 for b and g correspond to 0 for X and Y
    
show_image(decohered_pixels)

We could also use a different controlled operation for each qubit, which would copy a different coordinate. Here's what it would look like for a random choice.

In [10]:
# now the same, but the axis for the decoherence is randomly chosen for each pixel,
# instead of it always preserving the expectation value Z

# apply decoherence
decohered_pixels = {}
for pos in pixels:
    rand = random.random()
    if rand<1/3:
        decohered_pixels[pos] = ( pixels[pos][0], 128, 128, 255 )
    elif rand<2/3:
        decohered_pixels[pos] = ( 128, pixels[pos][1], 128, 255 )
    else:
        decohered_pixels[pos] = ( 128, 128, pixels[pos][2], 255 )
        
show_image(decohered_pixels)

Other ideas:

* This encoding would actually need many qubits for each pixel, all prepared in the same state. This is required for the decoding to be able to determine X, Y and Z, which are expectation values. Another copying method could then be to simply share out the qubits. With less samples to do statistics on, each copy would be degraded in comparison to the original.

* Since this encoding doesn't require entangled states, it could be run on a quantum computer as lots of single qubit jobs. This means that we could do it on current tech, even for arbitrarily large images. The only limit is the time we are willing to spend constantly submitting jobs!

